# Convolutional Neural Network

[Building powerful image classification models using very little data - Keras Blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)

### Importing the libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set


> Applying preprocessing only to the training set to prevent overfitting and apply data variations through geometric transformations(zoom,rotation,flipping etc.) of image, also called "**Image Augmentation**".

[Data Augmentation Code Snippet Copied from Keras Blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)

> <u>**rescale:**</u> a value by which we will multiply the data before any other processing. Our original images consist in RGB coefficients in the 0-255, but such values would be too high for our models to process (given a typical learning rate), so we target values between 0 and 1 instead by scaling with a 1/255 factor. <br>
<u>**shear_range**:</u> randomly applying shearing transformations (transformation in which all points along a given line remain fixed while other points are shifted parallel to by a distance proportional to their perpendicular distance from. Shearing a plane figure does not change its area or stretch dimensions, only changes interior angles. <br>
<u>**zoom_range**:</u> randomly zooming inside pictures <br>
horizontal_flip: randomly flipping half of the images horizontally --relevant when there are no assumptions of horizontal assymetry (e.g. real-world pictures). <br>
<u>**Example for Horizontal Flip:**</u><br>
"F" -> Mirror image of "F"<br>
"}" -> "{"<br>

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, # Apply feature scalling to each pixel by dividing the value by 255
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64), # Size of the input images; reduced to increase processing speed.
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

> <u>**flow_from_directory():**</u> this is a generator that will read pictures found in subfolders of 'dataset/training_set', and indefinitely generate batches of augmented image data.

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255) # Only Apply feature scalling to each pixel
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64), # all images will be resized to 64x64
                                            batch_size = 32,
                                            class_mode = 'binary') # since we use binary_crossentropy loss, we need binary labels

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [3]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution
#### Conv2D Layer Parameters:


> **filter**-> how many feature detector we want, <br>
  **kernel_size** -> easch feature detector matrix of 3x3 size ,<br>
  **activation** -> activation function used for convolutional layer ( Rectifier function )<br>
  **input_shape** -> input image er size, and '3' for colored image; "1" if black and white image  




In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# the model outputs 3D feature maps (height, width, features)

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())  # this converts our 3D feature maps to 1D feature vectors

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

> We end the model with a single unit and a sigmoid activation, which is perfect for a binary classification.

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

> To go with the output layer, we will also use the "**binary_crossentropy**" loss to train our model.

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 302s 1s/step - accuracy: 0.5646 - loss: 0.6769 - val_accuracy: 0.7020 - val_loss: 0.5995
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 93s 357ms/step - accuracy: 0.6826 - loss: 0.6067 - val_accuracy: 0.7275 - val_loss: 0.5692
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 231ms/step - accuracy: 0.7198 - loss: 0.5565 - val_accuracy: 0.7300 - val_loss: 0.5425
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 230ms/step - accuracy: 0.7275 - loss: 0.5290 - val_accuracy: 0.7595 - val_loss: 0.5041
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 231ms/step - accuracy: 0.7475 - loss: 0.5069 - val_accuracy: 0.7760 - val_loss: 0.4876
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 230ms/step - accuracy: 0.7672 - loss: 0.4779 - val_accuracy: 0.7725 - val_loss: 0.4834
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 230ms/step - accuracy: 0.7748 - loss: 0.4717 - val_accuracy: 0.7445 - val_loss: 0.5238
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 59s 230ms/step - accuracy: 0.7763 - loss: 0.4556 - val_a

## Part 4 - Making a single prediction

> tf.keras.utils.**load_img**(
    <u>**path**</u>,
    <u>**color_mode**</u>='rgb', <br>
    <u>**target_size**</u>=None,
    <u>**interpolation**</u>='nearest',
    <u>**keep_aspect_ratio**</u>=False
)

> <u>**Returns:**</u> A PIL Image instance.

[TensorFlow Documentation for "load_img()" method](https://www.tensorflow.org/api_docs/python/tf/keras/utils/load_img)

### BATCH SIZE & Expanding Image Array Dimension

> The **predict()** method expects the input to be **a batch of images**, even if you are only **predicting on a single image**. The **extra dimension at the beginning of images** are generally **for batch sizes**. Batch size represent **how many images you bundle to feed the CNN at once**. The **batch can also be a single image**, however it still **needs that extra dimension of 1 to show the size of batch**. If for example batch size is 5, your input array would be of shape (5, 150, 150, 3). <br>
The function **adds a dimension to a numpy array**. By **using "np.expand_dims(test_image, axis=0)"**, You load the image and get an **array of shape (150, 150, 3)**, after **calling "expand_dims()" in 0th axis**, you get **shape (1, 150, 150, 3)**. You are **adding an extra dimension at the beginning of the array**, effectively **creating a batch of size 1**. <br>
This allows the model to process the single image as if it were part of a larger batch.

[StackOverflow Explains Dimension Expanding of Image](https://stackoverflow.com/questions/62671394/how-to-interpret-expand-dims-while-predicting-a-uploaded-image-in-tensorflow)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_5.jpeg', target_size = (64, 64)) # cat
test_image = image.img_to_array(test_image) # coverts the PIL image instance format into an numpy array, in order to be accepted by predict() method.
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


{'cats': 0, 'dogs': 1}

In [ ]:
print( 'dog' if result[0][0] == 1 else 'cat' )

cat


In [ ]:
test_image = image.load_img('dataset/single_prediction/cat_or_dog_7.jpeg', target_size = (64, 64)) # dog
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print( 'dog' if result[0][0] == 1 else 'cat' )


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
dog


In [ ]:
test_image = image.load_img('dataset/single_prediction/cat_or_dog_4.jpg', target_size = (64, 64)) # cat
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print( 'dog' if result[0][0] == 1 else 'cat' )


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
cat


In [ ]:
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64)) # dog
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print( 'dog' if result[0][0] == 1 else 'cat' )


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
dog
